# Class 6: Word embeddings



## Preparation for class
---

So was sick friday so didn't get the chance to get this out before the weekend. So you really don't have to prepare anything, but I might be worth going through the gensim section code to get an overview before class and to make sure that everything work as intended.

## Gensim
---


You can install gensim simply using: 
```py
pip install gensim
```


### Using Gensim
to start us of we can download the word embeddings we will use for class using:

In [1]:
import gensim.downloader as api

# Pre-trained vectors based on Wikipedia 2014 + Gigaword, 
# 5.6B tokens, 400K vocab, 
# uncased (https://nlp.stanford.edu/projects/glove/).
# embedding size 50 (so fairly small, but easy to work with)
word_emb = api.load("glove-wiki-gigaword-300")

We can the parts of the word embedding class here, but I do encourage you to inspect it a bit more using e.g. `dir(word_emb)` or `help(word_emb)`:

In [72]:
print(type(word_emb))

# inspecting the vocabulary (keys in the dictionary)
vocab = word_emb.key_to_index.keys()
vocab = sorted(list(vocab)) # sort the vocab
print(vocab[:10])
print(vocab[51000: 51010])
print(vocab[-10:])

<class 'gensim.models.keyedvectors.KeyedVectors'>
['!', '!!', '!!!', '!!!!', '!!!!!', '!?', '!?!', '"', '#', '##']
['alemtuzumab', 'alemu', 'alemán', 'alemão', 'alen', 'alena', 'alencar', 'alencon', 'alendronate', 'alene']
['門下省', '魏博', '鳳翔', '법정동', '행정동', 'ﬁeld', 'ﬁgures', 'ﬁnds', 'ﬁrst', '￥']


We can now extract word embeddings for words in the vocabulary:

In [28]:
# fetch the word embedding for class
glass = word_emb["glass"]
print(type(glass))
print(glass.shape) # fairly small word embedding (50)

# check if word in vocabulary(50)

print("aarhus" in word_emb)


<class 'numpy.ndarray'>
(300,)
True


As you probably know from the lectures the similarity between word embedding is calculated using cosine similarity, which you might recall we implemented in the introduction week. We can use this to calculate the similarity between a couple of vectors here:

In [20]:
import numpy as np
def cosine(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

for w1, w2 in [("dog", "dog"), ("dog", "glass"), ("dog", "cat")]:
    sim = cosine(word_emb[w1], word_emb[w2])
    print(f"the similarity between {w1} and {w2} is {round(sim, 2)}")

# You can also calculate this using gensim:
word_emb.similarity('dog', 'glass')


the similarity between dog and dog is 1.0
the similarity between dog and glass is 0.27000001072883606
the similarity between dog and cat is 0.9200000166893005


0.27108675

You could imagine doing this for all words in the corpus to get the most similar words to `dog`, using gensim this can be done simply using:

In [23]:
print(word_emb.most_similar("dog", topn=10))

[('cat', 0.9218004941940308), ('dogs', 0.8513158559799194), ('horse', 0.7907583713531494), ('puppy', 0.7754920721054077), ('pet', 0.7724708318710327), ('rabbit', 0.7720814347267151), ('pig', 0.7490062117576599), ('snake', 0.7399188876152039), ('baby', 0.7395570278167725), ('bite', 0.7387937307357788)]


This function also helps you do analogies, *"woman is to ____ what man is to king"* or rephrased *king-man + woman = ?*:

In [25]:
# you can also do analogies using this function:
sim = word_emb.most_similar(positive = ["woman", "king"], negative=["man"], topn=1)
print(sim)

[('queen', 0.8523604273796082)]


Naturally the phrase *"king - man + woman = ?"* is nonsense, you can't do mathematics with words, but you can with word embeddings. So let us just go through what is going on here:
We start of taking the word embedding of king $w_{king}$ substract the word embedding for man $w_{man}$ and then add the word embedding for women $w_{woman}$, thus we have: 

$$w_{anology} = w_{king} - w_{man} + w_{woman}$$

Naturally it would be extremely coincidental that we were to have $w_{anology}$ being equal to $w_{queen}$, what we actually do is calculate the similarity between all vectors and $w_{anology}$ and find that $w_{queen}$ is the closest to that one, thus: 

$$w_{anology} \sim w_{queen}$$

Do note that we are working with vectors and that these are [cummutative](https://en.wikipedia.org/wiki/Commutative_property) (e.g. $v_1 + v_2 = v_2 + v_1$) thus it is just as valid to write:

$$w_{anology} = w_{king} + w_{woman} - w_{man} = w_{king} + (w_{woman} - w_{man})$$

Where you can interpret $w_{woman} - w_{man}$ as the *gender component* assuming that everything else about the usage of man and woman is similar i.e. they both refer to human, typically adult and seen as a neutral gendered description of a person. Even in simple cases such as this one it is trivially not true, *man* can even refer to a member of mankind (*god cares for all men*) an authority (*the man*) or can be used as an exclamation (*oh man*). 


<details>
    <summary> That sounds like a problem, what do we do to solve it? </summary>

Glad you ask. Well there are multiple and probably more than I don't know of. One way is to concieve the other meanings as noise in the *gender component* and one way to remove it is to average over mulitple gender components (another one could be $w_{girl} - w_{boy}$), you could probably imagine others as well. Another way is to disambiguate the word embeddings (by created the word embedding for each different meaning of the word), not a trivial task, but one place where you can start is including the part-of-speech tag and thus distingushing between e.g. *desert* as verb or *desert* as a noun. Word sense disambiguation is still an active area of reasearch.


</details>

<br /> 


### Understanding Super class initialization
We will go through this example at the start of class

In [8]:
class person():
    def __init__(self, name, year_of_birth):
        self.name = name
        self.age = 2021-year_of_birth
    
    def say_hello(self):
        print(f"Hello my name is {self.name} and I am {self.age} years old")

class researcher(person):
    def __init__(self, loc, employment):
        self.loc = loc
        self.employment = employment
        
    def say_hello(self):
        print(f"Hello I am currently a {self.employment}, at {self.loc}")


kenneth = researcher("Aarhus", "phd")
kenneth.say_hello()
# problem 1:
    # I want to reuse code from the init of person without copy pasting
# problem 2:
    # I want to be able to call the say_hello function of the parent class
    # e.g. kenneth.say_ordinary_hello()

Hello I am currently a phd, at Aarhus


What does the pytorch nn.Module init then do? Well from the [source code](https://pytorch.org/docs/stable/_modules/torch/nn/modules/module.html#Module) we find the following:

```py
    def __init__(self):
        """
        Initializes internal Module state, shared by both nn.Module and ScriptModule.
        """
        torch._C._log_api_usage_once("python.nn_module")

        self.training = True
        self._parameters = OrderedDict()
        self._buffers = OrderedDict()
        self._non_persistent_buffers_set = set()
        self._backward_hooks = OrderedDict()
        self._is_full_backward_hook = None
        self._forward_hooks = OrderedDict()
        self._forward_pre_hooks = OrderedDict()
        self._state_dict_hooks = OrderedDict()
        self._load_state_dict_pre_hooks = OrderedDict()
        self._modules = OrderedDict()
```

Which notably sets a couple of internal states (noticably self.training=True), and log it to the pytorch API.

## Exercises
---


you are not required to solve these tasks in any particular order. Exercise 6 is more challenging than the rest so if you are up for a challenge go for it.

1) Skim/read through the Gensim section to get an overview of how to work with word embeddings. 
2) What is to woman what man to doctor (according to the word embedding)? Is this problematic?
3) Discuss how you could use word embeddings to find pluralis of a word
  - apply it to three cases
  - plot these, does there seem to be a *plural component*?
3) Discuss how you would find the antonym of a word
  - apply it to three cases
  - plot these, does there seem to be a *antonym component*?
4) You can also use word embedding to detect the odd one out for instance which word does not belong in *Ross, Kenneth, Tim, glass*? You can do this simply using: `word_emb.doesnt_match("kenneth ross tim glass".split())`. Try this out on three samples.
  - Typically word embeddings are quite intuitive, discuss or write down your best answer to how one might find the odd one out?

<details>
    <summary> Answer </summary>

The function works by:

- taking the mean of all the word-embeddings.
- calculate the similarity (typically using cosine-similarity) from the mean to each word
- return the most dissimilar word (i.e. the one with the highest cosine-distance from that mean vector).

Do note that while your answer might not be the same it might still work as there are multiple ways of doing this. For instance you could also add up the distance between each word and every other word and see which one is the furthest away from all other points.

</details>

5) Which word embedding is closest to the word embedding of *corona*; *virus* or *beer*? Justify your answer before you test it using the word embedding.
  - What does this tell you about word embeddings?
6) Word embedding for token classification
  - We will here use token embeddding for classifying part-of-speech tags (i.e. whether a word is a noun or a verb), but this approach could just as well be used to classify whether a word is named entity or whether it is positive or negative. We will here use the English dependency treebank. In the utilities section below I have provided some functions to get you started, but I recommend solving it in 3 steps:
    1) transform the words to word embeddings
    2) Train a classifier to predict the pos-tag using e.g. logistic regression
    3) calculate the performance metrics on the held out test set (how well does it perform on unseen words?)

In [26]:
# 2) What is to woman what man is to doctor? 
sim = word_emb.most_similar(positive = ["woman","doctor"], negative = ['man'], topn=1)
print(sim)

sim = word_emb.most_similar(positive = "man") # restriction; it can never return the same vector as is it's input
print(sim)


[('physician', 0.6098570823669434)]
[('woman', 0.6998663544654846), ('person', 0.6443442106246948), ('boy', 0.6208277940750122), ('he', 0.5926738381385803), ('men', 0.5819568634033203), ('himself', 0.5810033082962036), ('one', 0.5779521465301514), ('another', 0.5721587538719177), ('who', 0.5703631639480591), ('him', 0.5670831203460693)]


In [52]:
# 3) How to find plural? 
sim = word_emb.most_similar(positive = ['dog','cats'], negative = ['cat'],topn=1)
print(sim)

sim = word_emb.most_similar(positive = ['men','woman'], negative = ['man'],topn=1)
print(sim)

sim = word_emb.most_similar(positive = ['rabbit','computers'], negative = ['computer'],topn=1)
print(sim)

sim = word_emb.most_similar(positive = ['houses','mouse'], negative = ['house'],topn=1)
print(sim)

[('dogs', 0.7930116057395935)]
[('women', 0.8786945939064026)]
[('rabbits', 0.6108177900314331)]
[('mice', 0.5166829228401184)]


In [69]:
# 3) Antonym 
sim = word_emb.most_similar(positive = ['sad','good'], negative=['bad'],topn=1)
print(sim)

sim = word_emb.most_similar(positive = ['warm','black'], negative='white', topn=1)
print(sim)

sim = word_emb.most_similar(positive = ['fact','fall'], negative='spring', topn=1)
print(sim)

[('happy', 0.5914756059646606)]
[('cool', 0.5465863943099976)]
[('indeed', 0.6467819809913635)]


In [13]:
# 4) No match
nomatch = word_emb.doesnt_match("Signe Sara Kenneth bottle".split())
print(nomatch)

nomatch = word_emb.doesnt_match("spoon fork knife glass".split())
print(nomatch)

nomatch = word_emb.doesnt_match("computer apple mouse table".split())
print(nomatch)


bottle
glass
table


In [21]:
# 5) which is closest to corona; beer or virus? 
sim = word_emb.similarity("corona","beer")
print(sim)
sim = word_emb.similarity("corona","virus")
print(sim)

0.1978268
0.083197795


## Utilities for tagging POS using word embeddings
This contain a bunch of utility functions for getting started using word embeddings for classifying part of speech tags.

Including a 1) logistic classifier (or a softmax classifier), 2) a function to load in the dataset as a list of token, pos-tag pairs, and 3) a code snippet for turning the label into one hot vectors.  

You can read more about the individual [pos-tags](https://universaldependencies.org/docsv1/en/pos/all.html) here.

In [9]:
import os

from spacy.training import Corpus
import spacy

# change this so it fits your file structure:
path = os.path.join("..", "data", "English Dependency Treeback", "en_ewt-ud-train.spacy")
# and use en_ewt-ud-dev.spacy and en_ewt-ud-test.spacy for the dev and test set.

def load_pos_data(path: str) -> list:
    corpus = Corpus(path)

    nlp = spacy.blank("en")

    def extract_token_label_pair(example):
        e = example.to_dict()
        tokens = e["token_annotation"]["ORTH"]
        entity_labels = e["token_annotation"]["POS"]
        return zip(tokens, entity_labels)
    train_samples = [(token, label) for example in corpus(nlp) for token, label in extract_token_label_pair(example)]

    return train_samples

train = load_pos_data(path)

In [10]:
# exploring the output
print(train[:10])

tokens, labels = list(zip(*train))
print(tokens[:10])

[('Al', 'PROPN'), ('-', 'PUNCT'), ('Zaman', 'PROPN'), (':', 'PUNCT'), ('American', 'ADJ'), ('forces', 'NOUN'), ('killed', 'VERB'), ('Shaikh', 'PROPN'), ('Abdullah', 'PROPN'), ('al', 'PROPN')]
('Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al')


In [11]:
# convert labels to one hot
import pandas as pd
df = pd.get_dummies(labels)
y = df.to_numpy() # One hot representation of the labels
df.head()

,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
import torch
import torch.nn as nn


class Logistic(nn.Module):
    def __init__(self, n_input_features: int, n_output_features: int):
        super().__init__()
        self.linear = nn.Linear(n_input_features, n_output_features)
        self.cost = nn.CrossEntropyLoss()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear(x)
    
    def fit(self, X: torch.Tensor, y: torch.Tensor, epochs = 1000):
        optimizer = torch.optim.AdamW(self.parameters())

        for epoch in range(epochs):
            # forward
            y_hat = self.forward(X)

            # backward
            loss = self.cost(y_hat, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # print
            if (epoch + 1) % 100 == 0:
                print(f"epoch: {epoch+1}, loss = {loss.item():.4f}")



## More on Word embeddings
---


Overview and introduction of word embeddings:

- Andrew NG have a small [video series](https://www.youtube.com/playlist?list=PLhWB2ZsrULv-wEM8JDKA1zk8_2Lc88I-s) on word embedding, which I highly recommend watching

More by Mikolov et al.:

- For the readings for class read on article by Mikolov et al. (2013) however the same year the team released an additional [paper](https://arxiv.org/abs/1310.4546) called *Distributed Representations of Words and Phrases and their Compositionality*. There is even a [video reading](https://www.youtube.com/watch?v=yexR53My2O4) of the paper which takes you through the main points.

Cross-lingual word embeddings:

- Wish I had the time to go through Cross-lingual word embeddings. The idea is that you overlay word embeddings for two languages and then this act as a way of translating between languages. It is a really cool application of word embeddings so if you feel like digging into it I recommend [this podcast episode](https://soundcloud.com/nlp-highlights/57-a-survey-of-cross-lingual-word-embedding-models-with-sebastian-ruder) (in general this podcast is great). You can also watch Kevin Clark (Manning's PhD student) give a [talk](https://www.youtube.com/watch?v=3wWZBGN-iX8&list=PLoROMvodv4rOhcuXMZkNm7j3fVwBBY42z&index=20) on how use word embeddings for translation without any training data.

